In [1]:
from knmi.downloader import KNMIDownloader
from knmi.database import KNMIDatabase
from knmi.processor import KNMIProcessor
from config import xlBasisfile, dbFile 
import pandas as pd

# Instantieer classes
db = KNMIDatabase(dbFile, xlBasisfile)
    
downloader = KNMIDownloader(db)
processor = KNMIProcessor(db)

# als de opgegeven database-file niet bestaat, wordt een nieuwe database aangemaakt
if not db.exists():
    db.create_database()
    db.fill_database()

# de database wordt (aan)gevuld met de meest recente data
#downloader.update_data()

#db.get_last_observations


In [2]:
processor.set_filter(locations= 'all', startyear= 1990, endyear=2020,startmonth=4,endmonth=9)

dfdp = processor.get_filtered_PDseries()

#dfdp['avg'] = dfdp.mean(axis=1)

#df_mean = dfdp.mean(axis=1)

dfdp

date
1990-04-01     0.000000
1990-04-02     0.000000
1990-04-03     1.746154
1990-04-04     3.361538
1990-04-05     2.492308
                ...    
2020-09-26    21.907692
2020-09-27    38.415385
2020-09-28    41.084615
2020-09-29    44.715385
2020-09-30    45.453846
Length: 4940, dtype: float64

## Snelheid testen

In [24]:
import sqlite3
import timeit

qry = '''SELECT date(Timestamp) AS date, value, quality 
                             FROM KNMI_data 
                             WHERE StationId = ? AND Parameter = ? AND Timestamp BETWEEN ? AND ?;
                       '''

with sqlite3.connect(db.db_path) as conn:
            
            startdate = pd.to_datetime('1900-01-01').to_julian_date()
            enddate = pd.to_datetime('2020-12-31').to_julian_date()

            cursor = conn.cursor()
            t0 = timeit.default_timer()
            result1 = cursor.execute(qry, (1,'P',startdate, enddate)).fetchall()
            t1 = timeit.default_timer()
            result2 = pd.read_sql_query(qry,conn, params= (1,'P',startdate,enddate))
            t2 = timeit.default_timer()

            print(f'Time 1: {t1-t0}')
            print(f'Time 2: {t2-t1}')
            

Time 1: 0.12456239999983154
Time 2: 0.13187779999998384


### 

In [29]:
qry2 = '''SELECT date(Timestamp) AS date, StationId, Parameter, value, quality 
                             FROM KNMI_data 
                             WHERE Timestamp BETWEEN ? AND ?;
                       '''

with sqlite3.connect(db.db_path) as conn:
            
            startdate = pd.to_datetime('1900-01-01').to_julian_date()
            enddate = pd.to_datetime('2020-12-31').to_julian_date()

            cursor = conn.cursor()
            t0 = timeit.default_timer()
            result1 = cursor.execute(qry2, (startdate, enddate)).fetchall()
            t1 = timeit.default_timer()
            result2 = pd.read_sql_query(qry2,conn, params = (startdate,enddate))
            t2 = timeit.default_timer()

            print(f'Time 1: {t1-t0}')
            print(f'Time 2: {t2-t1}')


Time 1: 1.0279921999999715
Time 2: 1.1640588000000207


In [42]:
qry3 =  '''SELECT StationId, Name, KNMI_data.Parameter as Param, date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date 
                            FROM KNMI_data INNER JOIN KNMI_stations USING (StationId)
                            GROUP BY Param, Name;
                     '''
with sqlite3.connect(db.db_path) as conn:
    res3 = pd.read_sql_query(qry3,conn)

res3.set_index('StationId').sort_index()

,Name,Param,first_date,last_date
StationId,,,,
1,De Bilt,P,1898-01-01,2025-11-30
2,De Kooy,P,1958-04-01,2025-11-30
3,Den Helder,P,1851-01-01,1972-12-31
4,Groningen,P,1847-01-01,2025-11-30
5,Heerde,P,1892-08-31,2025-11-30
6,Hoofddorp,P,1866-12-01,2025-11-30
7,Hoorn,P,1883-04-01,2025-11-30
8,Kerkwerve,P,1878-04-02,2025-11-30
9,Oudenbosch,P,1888-02-02,2025-11-30


In [74]:
#qry4 =  '''SELECT LocationName, KNMI_data.Parameter , StationId, date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date
#                            FROM KNMI_data 
#                              RIGHT JOIN (P13_Locations RIGHT JOIN locations_stations USING (LocationId))
#                             USING (StationID)
#                            ;
#                     '''

qry4 =  '''SELECT LocationId, locations_stations.StationId, date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date
                            FROM locations_stations 
                              INNER JOIN KNMI_data ON locations_stations.StationId = KNMI_data.StationId
                              GROUP BY LocationId, locations_stations.StationId;'''

qry5 = '''SELECT LocationName, locations_stations.LocationId, locations_stations.Parameter, date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date 
          FROM P13_locations
           INNER JOIN (locations_stations INNER JOIN KNMI_data ON locations_stations.Parameter = KNMI_data.Parameter AND locations_stations.StationId=KNMI_data.StationId) B
            ON P13_locations.LocationId = B.LocationId
          GROUP BY P13_locations.LocationId, B.Parameter;'''

with sqlite3.connect(db.db_path) as conn:
    res4 = pd.read_sql_query(qry5,conn)

res4

,LocationName,LocationId,Parameter,first_date,last_date
0,De Bilt,1,E,1957-07-01,2025-12-21
1,De Bilt,1,P,1898-01-01,2025-11-30
2,De Kooy,2,E,1957-07-01,2025-12-21
3,De Kooy,2,P,1851-01-01,2025-11-30
4,Groningen,3,E,1957-07-01,2025-12-21
5,Groningen,3,P,1847-01-01,2025-11-30
6,Heerde,4,E,1957-07-01,2025-12-21
7,Heerde,4,P,1892-08-31,2025-11-30
8,Hoofddorp,5,E,1957-07-01,2025-12-21
9,Hoofddorp,5,P,1866-12-01,2025-11-30
